In [19]:

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# UQpy imports
from UQpy.scientific_machine_learning.layers import Dropout
from UQpy.scientific_machine_learning.neural_networks import FeedForwardNeuralNetwork
from UQpy.scientific_machine_learning.trainers import Trainer
import logging

logger = logging.getLogger("UQpy")
logger.setLevel(logging.INFO)

torch.manual_seed(0);

In [20]:
width = 20
network = nn.Sequential(
    nn.Linear(1, width),
    nn.ReLU(),
    nn.Linear(width, width),
    Dropout(drop_rate=0.1),
    nn.ReLU(),
    nn.Linear(width, width),
    Dropout(drop_rate=0.1),
    nn.ReLU(),
    nn.Linear(width, width),
    Dropout(drop_rate=0.1),
    nn.ReLU(),
    nn.Linear(width, 1),
)
model = FeedForwardNeuralNetwork(network)

In [21]:
class SinusoidalDataset(Dataset):
    def __init__(self, n_samples=20, noise_std=0.05):
        self.n_samples = n_samples
        self.noise_std = noise_std
        self.x = torch.linspace(-1, 1, n_samples).reshape(-1, 1)
        self.y = torch.tensor(0.4 * np.sin(4 * self.x) + 0.5 * np.cos(12 * self.x) +
                              np.random.normal(0, self.noise_std, self.x.shape), dtype=torch.float)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, item):
        return self.x[item], self.y[item]

train_data = DataLoader(SinusoidalDataset(), batch_size=20, shuffle=True)

In [22]:
# %% Train with dropout layers inactive
model.eval()  # this deactivates the dropout layers
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
trainer = Trainer(model, optimizer)

In [ ]:
print("Starting Training...", end="")
trainer.run(train_data=train_data, epochs=50000)
print("done")

[INFO] - 2024-05-16 12:38:13,449 - UQpy: Scientific Machine Learning: Beginning training FeedForwardNeuralNetwork
[INFO] - 2024-05-16 12:38:13,452 - UQpy: Scientific Machine Learning: Epoch 1 / 50,000 Train Loss 0.2091282159090042
[INFO] - 2024-05-16 12:38:13,455 - UQpy: Scientific Machine Learning: Epoch 2 / 50,000 Train Loss 0.20515871047973633
[INFO] - 2024-05-16 12:38:13,457 - UQpy: Scientific Machine Learning: Epoch 3 / 50,000 Train Loss 0.20027372241020203
[INFO] - 2024-05-16 12:38:13,459 - UQpy: Scientific Machine Learning: Epoch 4 / 50,000 Train Loss 0.21357782185077667
[INFO] - 2024-05-16 12:38:13,460 - UQpy: Scientific Machine Learning: Epoch 5 / 50,000 Train Loss 0.19905821979045868
[INFO] - 2024-05-16 12:38:13,462 - UQpy: Scientific Machine Learning: Epoch 6 / 50,000 Train Loss 0.19868406653404236
[INFO] - 2024-05-16 12:38:13,464 - UQpy: Scientific Machine Learning: Epoch 7 / 50,000 Train Loss 0.19796334207057953
[INFO] - 2024-05-16 12:38:13,465 - UQpy: Scientific Machine L

Starting Training...

[INFO] - 2024-05-16 12:38:13,650 - UQpy: Scientific Machine Learning: Epoch 171 / 50,000 Train Loss 0.19632044434547424
[INFO] - 2024-05-16 12:38:13,651 - UQpy: Scientific Machine Learning: Epoch 172 / 50,000 Train Loss 0.18796631693840027
[INFO] - 2024-05-16 12:38:13,651 - UQpy: Scientific Machine Learning: Epoch 173 / 50,000 Train Loss 0.1793767660856247
[INFO] - 2024-05-16 12:38:13,652 - UQpy: Scientific Machine Learning: Epoch 174 / 50,000 Train Loss 0.17897555232048035
[INFO] - 2024-05-16 12:38:13,653 - UQpy: Scientific Machine Learning: Epoch 175 / 50,000 Train Loss 0.1876375526189804
[INFO] - 2024-05-16 12:38:13,654 - UQpy: Scientific Machine Learning: Epoch 176 / 50,000 Train Loss 0.15658387541770935
[INFO] - 2024-05-16 12:38:13,655 - UQpy: Scientific Machine Learning: Epoch 177 / 50,000 Train Loss 0.17298294603824615
[INFO] - 2024-05-16 12:38:13,656 - UQpy: Scientific Machine Learning: Epoch 178 / 50,000 Train Loss 0.17063501477241516
[INFO] - 2024-05-16 12:38:13,657 - UQpy: S

In [ ]:
# Plotting Results
x = SinusoidalDataset().x
y = SinusoidalDataset().y
x_val = torch.linspace(-1, 1, 1000).view(-1,1)
y_val = (0.4 * torch.sin(4 * x_val) + 0.5 * torch.cos(12 * x_val))
pred_val = model(x_val)

In [ ]:
#%% Plot the deterministic model estimates
fig, ax = plt.subplots()
ax.plot(
    x.detach().numpy(),
    y.detach().numpy(), '*',
    label="Data",
    color="tab:blue",
)
ax.plot(
    x_val.detach().numpy(),
    pred_val.detach().numpy(),
    label="Final Prediction",
    color="tab:orange",
)
ax.plot(
    x_val.detach(), y_val.detach(),
    label="Exact",
    color="black",
    linestyle="dashed",
)

ax.set_title("Predictions")
ax.set(xlabel="x", ylabel="f(x)")
ax.legend()

In [ ]:
train_loss = trainer.history["train_loss"].detach().numpy()
fig, ax = plt.subplots()
ax.plot(train_loss)
ax.set_title("Deterministic training Loss")
ax.set(xlabel="Epoch", ylabel="Loss")
plt.show()
# %%
model.train() # this activates the dropout layers
n = 10000
samples = torch.zeros(len(x_val), n)
for i in range(n):
    samples[:, i] = model(x_val).detach().squeeze()
variance = torch.var(samples, dim=1)
standard_deviation = torch.sqrt(variance)


In [ ]:
#%%
x_plot = x.squeeze().detach().numpy()
mu =  torch.mean(samples, dim=1)
sigma = standard_deviation.squeeze().detach().numpy()
fig, ax = plt.subplots()
ax.plot(x_val, mu, label="$\mu$")
ax.plot(x_val, y_val.detach().numpy(), label="Exact", color="black", linestyle="dashed")
ax.fill_between(
    x_val.view(-1), mu - (3 * sigma), mu + (3 * sigma), label="$\mu \pm 3\sigma$,", alpha=0.3
)
ax.set_title("Bayesian Neural Network $\mu \pm 3\sigma$")
ax.set(xlabel="x", ylabel="f(x)")
ax.legend()

plt.show()
# %%
